In [19]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller

In [2]:
train_test = pd.read_csv('../Bejing Data/Train_test.csv')
train_test

,date,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0
...,...,...,...,...,...,...,...,...,...
35034,2013-12-31 18:00:00,15.0,-19,8.0,1013.0,NW,109.95,0,0
35035,2013-12-31 19:00:00,22.0,-19,7.0,1013.0,NW,114.87,0,0
35036,2013-12-31 20:00:00,18.0,-21,7.0,1014.0,NW,119.79,0,0
35037,2013-12-31 21:00:00,23.0,-21,7.0,1014.0,NW,125.60,0,0


In [3]:
train_test["wnd_dir"].value_counts()

SE    12068
NW    11673
cv     7305
NE     3993
Name: wnd_dir, dtype: int64

In [4]:
train_test["wnd_dir"]=pd.factorize( train_test["wnd_dir"])[0]

In [5]:
train_test

,date,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,0,1.79,0,0
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,0,2.68,0,0
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,0,3.57,0,0
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,0,5.36,1,0
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,0,6.25,2,0
...,...,...,...,...,...,...,...,...,...
35034,2013-12-31 18:00:00,15.0,-19,8.0,1013.0,2,109.95,0,0
35035,2013-12-31 19:00:00,22.0,-19,7.0,1013.0,2,114.87,0,0
35036,2013-12-31 20:00:00,18.0,-21,7.0,1014.0,2,119.79,0,0
35037,2013-12-31 21:00:00,23.0,-21,7.0,1014.0,2,125.60,0,0


In [6]:
train_test=train_test.drop("date",axis=1)

In [7]:
train_test["wnd_dir"].value_counts()

0    12068
2    11673
1     7305
3     3993
Name: wnd_dir, dtype: int64

In [8]:
x=train_test.drop("pollution",axis=1)
y=train_test["pollution"]

In [9]:
y

0        129.0
1        148.0
2        159.0
3        181.0
4        138.0
         ...  
35034     15.0
35035     22.0
35036     18.0
35037     23.0
35038     20.0
Name: pollution, Length: 35039, dtype: float64

In [10]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=22)

In [11]:
# pip install pmdarima

In [12]:
from pmdarima.arima import auto_arima

In [13]:
y_train

20906     93.0
34961     20.0
16877    229.0
19513     40.0
26552    278.0
         ...  
20971     17.0
23030      0.0
24932     38.0
31680     49.0
29484     93.0
Name: pollution, Length: 28031, dtype: float64

In [24]:
adft = adfuller(y,autolag="AIC")
output_df= pd.DataFrame({"Values":[adft[0],adft[1],adft[2],adft[3], adft[4]['1%'], adft[4]['5%'], adft[4]['10%']], "Metric":["Test Statistics","p-value","No. of lags used","Number of observations used", 
                                                        "critical value (1%)", "critical value (5%)", "critical value (10%)"]})
print(output_df)

         Values                       Metric
0    -19.095143              Test Statistics
1      0.000000                      p-value
2     52.000000             No. of lags used
3  34986.000000  Number of observations used
4     -3.430537          critical value (1%)
5     -2.861623          critical value (5%)
6     -2.566814         critical value (10%)


In [14]:
model = auto_arima(y_train,x_train,trace=True, error_action='ignore', suppress_warnings=True)
model.fit(y_train)
forecast = model.predict(n_periods=len(y_test))
forecast = pd.DataFrame(forecast,index = y_test.index,columns=['Prediction'])

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=325821.126, Time=10.01 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=325814.497, Time=6.44 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=325816.147, Time=4.46 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=325816.148, Time=5.29 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=325812.768, Time=16.80 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=inf, Time=40.60 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0]          
Total fit time: 83.648 seconds


In [15]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                28031
Model:                        SARIMAX   Log Likelihood             -176380.256
Date:                Thu, 03 Mar 2022   AIC                         352762.511
Time:                        18:54:41   BIC                         352770.753
Sample:                             0   HQIC                        352765.164
                              - 28031                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2       1.71e+04     95.711    178.652      0.000    1.69e+04    1.73e+04
===================================================================================
Ljung-Box (L1) (Q):                   0.67   Jarque-Bera (JB):             48565.03
Prob(Q):                              0.41   Prob(JB):                         0.00
Heteroskedasticity (H):               0.98   Skew:                             1.83
Prob(H) (two-sided):                  0.31   Kurtosis:                         8.31
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [16]:
# Accuracy Scor
from sklearn.metrics import accuracy_score
accuracy_score(y,y_pred)




NameError: name 'y_pred' is not defined

In [17]:
# RMSE cell
from math import sqrt
from sklearn.metrics import mean_squared_error
rms = sqrt(mean_squared_error(y_test,forecast))
print("RMSE: ", rms)

RMSE:  131.9878099048178
